In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Flatten, Dense,GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
url_data = pd.read_csv('/content/drive/MyDrive/javas_project/data/malicious_phish.csv')

In [ ]:
lb_make = LabelEncoder()
url_data["type_code"] = lb_make.fit_transform(url_data["type"])
url_data["type_code"].value_counts()

urls = url_data['url']
labels = url_data['type_code']

In [ ]:
data = url_data.drop(columns=['type'])

data['url'] = data['url'].drop_duplicates().reset_index(drop=True)

data = data.dropna()

data.info()

In [ ]:
urls = url_data['url']
labels = url_data['type_code']

tokenizer = Tokenizer()
# tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(urls)
total_words = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(urls)

padded_sequences = pad_sequences(sequences)

num_classes = len(np.unique(labels))
labels_one_hot = to_categorical(labels, num_classes)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels_one_hot, test_size=0.3, random_state=42)

In [ ]:
best_model_path = '/content/drive/MyDrive/javas_project/data/best_model1/dm_{epoch:02d}_{val_accuracy:0.2f}.hdf5'
bmc = ModelCheckpoint(filepath=best_model_path,verbose=1,save_best_only=True,monitor='val_accuracy')
bes = EarlyStopping(monitor='val_accuracy',verbose=1,patience=10)

In [ ]:
model = Sequential()

model.add(Embedding(input_dim=total_words, output_dim=128, input_length=padded_sequences.shape[1]))

model.add(LSTM(units=100, return_sequences=True))

model.add(GlobalMaxPooling1D())
# model.add(GlobalAveragePooling1D())

model.add(Dense(units=4, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))

model.add(Dense(units=num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=100, validation_split=0.2, batch_size=128, callbacks=[bmc,bes])
model.evaluate(X_test,y_test)